In [1]:
#r "nuget:Microsoft.Spark,0.12.1"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Spark version 0.12.1

In [2]:
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

var inputFile = @"c:\users\bindeshv\projects_smaller.csv";



In [3]:
SparkSession spark = SparkSession.Builder()
                .AppName("Spark.NET Batch")
                .GetOrCreate();

[2020-09-06T09:16:13.4089170Z] [MININT-GLN1L3K] [Info] [ConfigurationService] 'DOTNETBACKEND_PORT' environment variable is not set.
[2020-09-06T09:16:13.4143390Z] [MININT-GLN1L3K] [Info] [ConfigurationService] Using port 5567 for connection.
[2020-09-06T09:16:13.4147703Z] [MININT-GLN1L3K] [Info] [JvmBridge] JvMBridge port is 5567


In [7]:
 DataFrame projectsDF = spark.Read()
                .Schema("id INT, url STRING, owner_id INT, name STRING, descriptor STRING, language STRING, created_at STRING, forked_from INT, deleted STRING, updated_at STRING")
                .Csv("c:\\users\\bindeshv\\DataBig\\projects_smaller.csv");

In [8]:
projectsDF.Show()

+----+--------------------+--------+--------------------+--------------------+--------+---------------+-----------+-------+-------------------+
|  id|                 url|owner_id|                name|          descriptor|language|     created_at|forked_from|deleted|         updated_at|
+----+--------------------+--------+--------------------+--------------------+--------+---------------+-----------+-------+-------------------+
|   1|https://api.githu...|       1|           ruote-kit|RESTful wrapper f...|    Ruby|12/8/2009 10:17|          2|      0|     11/5/2015 1:15|
|null|                null|    null|                null|                null|    null|           null|       null|   null|               null|
|null|                null|    null|                null|                null|    null|           null|       null|   null|               null|
|   4|https://api.githu...|      24|             basemap|                null|     C++|6/14/2012 14:14|          3|      1|0000-00-00 00

In [9]:
DataFrameNaFunctions dropEmptyProjects = projectsDF.Na();
DataFrame cleanedProjects = dropEmptyProjects.Drop("any");
//drop unneccessary columns 
cleanedProjects = cleanedProjects.Drop("id", "owner_id", "url");
cleanedProjects.Show();


+--------------------+--------------------+-----------+----------------+-----------+-------+-------------------+
|                name|          descriptor|   language|      created_at|forked_from|deleted|         updated_at|
+--------------------+--------------------+-----------+----------------+-----------+-------+-------------------+
|           ruote-kit|RESTful wrapper f...|       Ruby| 12/8/2009 10:17|          2|      0|     11/5/2015 1:15|
|           cocos2d-x|Port of cocos2d-i...|        C++| 3/12/2012 16:48|          6|      0|   10/22/2015 17:36|
|           cocos2d-x|Port of cocos2d-i...|          C| 4/23/2012 10:20|          6|      0|    11/1/2015 17:32|
|       rake-compiler|Provide a standar...|       Ruby|  8/1/2012 18:33|   14556189|      0|    11/3/2015 19:30|
|    cobertura-plugin|Jenkins cobertura...|       Java| 7/26/2012 18:46|     193522|      0|    11/1/2015 19:55|
|     scala-vs-erlang|A performance tes...|     Erlang|12/25/2011 13:51|    1262879|      0|    

In [11]:
//get the average number of times each language has been forked
DataFrame groupedDF = cleanedProjects.GroupBy("language")
                                    .Agg(Avg(cleanedProjects["forked_from"]));
groupedDF.Show();

+------------+------------------+
|    language|  avg(forked_from)|
+------------+------------------+
|          C#|        914767.625|
|    Makefile|         3611689.0|
|        Rust|             288.0|
|  JavaScript| 240718.6494845361|
|  Emacs Lisp| 660.3333333333334|
|          \N| 1995.095238095238|
|        Perl|          298380.0|
|      Puppet|          253680.5|
|      Erlang|         1262879.0|
|         C++| 448911.1538461539|
|      Groovy|            1049.0|
|ActionScript|        2474502.75|
|           D|             458.0|
|           C|1576.4705882352941|
| Objective-C|2281.4761904761904|
|      Augeas|             680.0|
|        VimL|             725.0|
|        HTML|             486.0|
|        HaXe|             829.0|
|   Smalltalk|             394.0|
+------------+------------------+
only showing top 20 rows



In [12]:
 //sort on most forked languages first 
groupedDF.OrderBy(Desc("avg(forked_from)")).Show();

+------------+------------------+
|    language|  avg(forked_from)|
+------------+------------------+
|      Racket|       1.2550711E7|
|    Makefile|         3611689.0|
|ActionScript|        2474502.75|
|      Erlang|         1262879.0|
|          C#|        914767.625|
|         PHP| 617219.4333333333|
|         C++| 448911.1538461539|
|        Ruby|349311.23214285716|
|        Perl|          298380.0|
|      Puppet|          253680.5|
|  JavaScript| 240718.6494845361|
|        Java|      149923.71875|
|      Python| 76043.26190476191|
|       Shell|           13276.6|
| Objective-C|2281.4761904761904|
|          \N| 1995.095238095238|
|           C|1576.4705882352941|
|       Scala|            1358.5|
|      Groovy|            1049.0|
|        HaXe|             829.0|
+------------+------------------+
only showing top 20 rows

